In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from datetime import datetime
from utils import BNN
import json
import openturns as ot
import seaborn as sns

In [ ]:
# Definition de la fonction de test
def f(x, sigma):
    epsilon = np.random.randn(*x.shape) * sigma
    return 10*np.sin(2*np.pi*x) + epsilon

# def f(x, sigma):
#     epsilon = np.random.randn(*x.shape) * sigma
#     return np.cos(x) + epsilon


# Paramétrage des données d'entrainements
train_size = 100
noise = 0.1
# Pour sin
xmin = -0.5
xmax = 0.5

zoomfig = 2
xminBig = xmin - zoomfig * (xmax - xmin)
xmaxBig = xmax + zoomfig * (xmax - xmin)

# # Pour cos
# xmin = -3.5
# xmax = 3.5

XBig = np.linspace(xminBig, xmaxBig, train_size *
                   (2*zoomfig + 1)).reshape(-1, 1)
X = XBig[train_size * zoomfig: train_size * zoomfig + train_size]

# X = np.linspace(-np.pi, np.pi, train_size).reshape(-1, 1)

y = f(X, sigma=noise)
y_true = f(X, sigma=0.0)


# On affiche tout ça
plt.close()
plt.scatter(X, y, marker='+', label='Training data')
plt.plot(X, y_true, label='Truth')
plt.title('Noisy training data and ground truth')
plt.legend()
plt.show()

In [ ]:
XTBig = torch.Tensor(XBig).reshape(XBig.shape)
XT = torch.Tensor(X).reshape(X.shape)
y = torch.Tensor(y).reshape(y.shape)
y_true = torch.Tensor(y_true).reshape(y_true.shape)

In [ ]:
# Hypperparamètres
N = 20000
lmax = 20
P0 = 0.1
epsilon = 1
sigma_0 = (lmax + 1) * 0.1
ns = [1, 15, 15, 1]

# Temperature initiale
# T = np.sum(1 / np.log(2 + np.arange(lmax))) + 15
Temp = 30

# Distance qui va être utiliser pour évaluer
# la dissimilarité entre les prédictions y_hat et la réponse y
pdist = 2

# Ici on commence l'algorithm BNN-ABC-SS

# Notre modèle
myModel = BNN.FNN(ns)

NP0 = int(N*P0)
invP0 = int(1/P0)
myModelSize = BNN.modelSize(ns)


In [ ]:
# thetaPrior = ot.Normal(np.zeros(myModelSize), np.ones(myModelSize))
# qDist = ot.Normal(np.zeros(myModelSize), np.ones(myModelSize) * sigma_0)

In [ ]:
thetaPrior = ot.Normal(0, sigma_0)
# qDist = ot.Normal(0, sigma_0)

In [ ]:
thetas = torch.randn(myModelSize, N) * sigma_0
# pthetas = torch.Tensor(thetaPrior.computePDF(thetas.t().detach().numpy()))

In [ ]:
# On evalue tout ça
y_hats = torch.concat(
    tuple([myModel.update_weights(thetas[:, i]).forward(XT) for i in range(0, N)]), 1)

# On calcul la dissimilarité
rho_n = torch.cdist(y_hats.t(), y.t(), p=pdist)

# On fixe un petit truc qui va être utile après
# c'est en fait le pas
sigma_j = sigma_0

# Relative learning rate
rhoMin = []
rhoMax = []


# Iteration
l_eps = []
# for j in range(0, l):
j = 0

In [ ]:
# plt.ion()

# while (rho_n[0, 0] > epsilon):

#     # On trie les erreurs et on mets les poids dans
#     # l'ordre croissant des érreurs qu'ils produisent
#     rho_n, indices = torch.sort(rho_n, 0)

#     rhoMin.append(str(np.float32(torch.min(rho_n).detach())))
#     rhoMax.append(str(np.float32(torch.max(rho_n).detach())))

#     thetas = thetas[:, indices.t()[0]]

#     epsilon_j = rho_n[NP0]

#     # # Lr
#     # rho_nOld = rho_n

#     # Ici on a un échantillion de taille NP0 et on veut
#     # en créer N à partir de cette échantillion en fesant
#     # (invPO - 1) pas
#     thetasSeeds = thetas[:, :NP0]
#     rhoSeeds = rho_n[:NP0]

#     # Réglage de sigma_j
#     sigma_j = sigma_0

#     #
#     thetas = thetasSeeds
#     rho_n = rho_n[:NP0]

#     # Réglage de la température
#     TempCur = Temp
#     for g in range(invP0 - 1):
#         # for debugging purposes
#         l_eps.append(epsilon_j)
#         # thetasResamples = torch.normal(thetasSeeds, sigma_j)
#         thetasResamples = thetasSeeds + \
#             torch.randn(thetasSeeds.shape) * sigma_j

#         # On evalue les erreurs
#         y_hatsResamples = torch.concat(
#             tuple([myModel.update_weights(thetasResamples[:, i]).forward(XT) for i in range(0, NP0)]), 1)
#         rhoResamples = torch.cdist(y_hatsResamples.t(), y.t(), p=pdist)

#         # On évalue l'amélioration
#         deltaRho = rhoResamples - rhoSeeds
#         deltaTheta = thetasResamples - thetasSeeds

#         # Mise à jour de la chaine de markovs
#         pthetasSeeds = torch.Tensor(thetaPrior.computePDF(thetasSeeds.t().detach().numpy()))
#         pthetasResamples = torch.Tensor(thetaPrior.computePDF(thetasResamples.t().detach().numpy()))
#         # qSeedsResamples = torch.Tensor(qDist.computePDF((thetasSeeds - thetasResamples).t().detach().numpy()))
#         # qResamplesSeeds = torch.Tensor(qDist.computePDF((thetasResamples - thetasSeeds).t().detach().numpy()))
#         IsInNepsj = (rhoResamples <= epsilon_j).float()

#         # alpha = torch.minimum(torch.ones(NP0,1), ((pthetasResamples * qResamplesSeeds) / (pthetasSeeds * qSeedsResamples)) * IsInNepsj)
#         alpha = torch.minimum(torch.ones(NP0,1), ((pthetasResamples) / (pthetasSeeds)) * IsInNepsj)

#         avanceOuNon = torch.bernoulli(1 - alpha)

#         # ThetaFinal = ThetaResample - mask * (ThetaResample - ThetaSeed)
#         mask = torch.diag(avanceOuNon[:, 0].float())

#         # Mise à jour
#         thetasSeeds = thetasResamples - torch.matmul(deltaTheta, mask)
#         thetas = torch.concatenate((thetas, thetasSeeds), 1)
#         rhoSeeds = rhoResamples - torch.matmul(mask, deltaRho)
#         rho_n = torch.concatenate((rho_n, rhoSeeds))

#         # Réglage de la température
#         # TempCur = max(Temp / np.log(2 + g), TempMin)
#         TempCur = Temp / np.log(2 + g)

#     j += 1
#     if (j >= lmax):
#         break

#     # %matplotlib
#     plt.clf()
#     BNN.plotTubeMedianBig(XT, XTBig, y, thetas, ns, 1)
#     plt.title("Epoch {}| T {}".format(j, TempCur))
#     plt.show()
#     plt.pause(0.1)

In [ ]:
plt.close()
plt.ion()

while (rho_n[0, 0] > epsilon):

    # On trie les erreurs et on mets les poids dans
    # l'ordre croissant des érreurs qu'ils produisent
    rho_n, indices = torch.sort(rho_n, 0)

    rhoMin.append(str(np.float32(torch.min(rho_n).detach())))
    rhoMax.append(str(np.float32(torch.max(rho_n).detach())))

    thetas = thetas[:, indices.t()[0]]

    epsilon_j = rho_n[NP0]

    # Ici on a un échantillion de taille NP0 et on veut
    # en créer N à partir de cette échantillion en fesant
    # (invPO - 1) pas
    thetasSeeds = thetas[:, :NP0]
    rhoSeeds = rho_n[:NP0]

    # Réglage de sigma_j
    sigma_j = sigma_0 - 0.1 * lmax

    #
    thetas = thetasSeeds
    rho_n = rho_n[:NP0]

    # Réglage de la température
    TempCur = Temp
    for g in range(invP0 - 1):
        # for debugging purposes
        l_eps.append(epsilon_j)
        # thetasResamples = torch.normal(thetasSeeds, sigma_j)
        thetasResamples = thetasSeeds + \
            torch.randn(thetasSeeds.shape) * sigma_j

        # redimensionement des thetas
        tThetasSeeds = thetasSeeds.t().ravel().reshape(myModelSize * NP0, 1)
        tThetasResamples = thetasResamples.t().ravel().reshape(myModelSize * NP0, 1)

        # Calcul des probabilité p(theta^{(n-1)}) et p(theta')
        ptThetasSeeds = torch.Tensor(
            thetaPrior.computePDF(tThetasSeeds.detach().numpy()))
        ptThetasResamples = torch.Tensor(
            thetaPrior.computePDF(tThetasResamples.detach().numpy()))

        rj = (ptThetasResamples / ptThetasSeeds)

        sj = torch.minimum(torch.ones(myModelSize * NP0, 1), rj)
        bj = torch.bernoulli(sj)

        thetasNow = (bj * tThetasResamples + (1 - bj) *
                     tThetasSeeds).reshape(NP0, myModelSize).t()
        # On evalue les erreurs
        y_hatsNow = torch.concat(
            tuple([myModel.update_weights(thetasNow[:, i]).forward(XT) for i in range(0, NP0)]), 1)
        rhoNow = torch.cdist(y_hatsNow.t(), y.t(), p=pdist)

        paramTemp = 1 - 1 / (1 + torch.exp( - (rhoNow - epsilon_j)/ Temp))
        mask = torch.diag((torch.bernoulli(paramTemp)).ravel().float())

        # mask = torch.diag((rhoNow <= epsilon_j).ravel().float())
        rhoVal = mask @ rhoNow + (torch.eye(NP0) - mask) @ rhoSeeds
        thetaVal = thetasNow @ mask + thetasSeeds @ (torch.eye(NP0) - mask)

        # Mise à jour
        thetasSeeds = thetaVal
        thetas = torch.concatenate((thetas, thetasSeeds), 1)
        rhoSeeds = rhoVal
        rho_n = torch.concatenate((rho_n, rhoSeeds))

        # Réglage de la température
        TempCur = Temp / np.log(2 + g)

    j += 1
    if (j >= lmax):
        break

    %matplotlib
    plt.clf()
    BNN.plotTubeMedianBig(XT, XTBig, y, thetas, ns, 1)
    plt.title("Epoch {}".format(j))
    plt.show()
    plt.pause(0.1)


In [ ]:
# plt.clf()
# # BNN.plotTubeMedian(XT, y, thetas, ns)
# BNN.plotTubeMedianBig(XT, XTBig, y, thetas, ns, 1)
# plt.xlim(-1.5, 1.5)
# plt.ylim(-20, 30)

# # save the figure
# # datetime object containing current date and time
# now = datetime.now()
# # dd-mm-YY_H:M:S
# dt_string = now.strftime("%d-%m-%Y_%H-%M-%S")

# plt.savefig('plots/plotCos-{}.png'.format(dt_string))

In [ ]:
# # Sauvegarde des poids
# f = open('thetas/Thetas{}.pt'.format(dt_string), 'w')
# f.close()
# torch.save(thetas, 'thetas/Thetas{}.pt'.format(dt_string))

# # Sauvegarde des hypperparametres
# hyperparameters = {"N": N, "lmax": lmax, "P0": P0, "epsilon": epsilon,
#                    "sigma_0": sigma_0, "structure": ns, "pdist": pdist}

# f = open('hyperparameters/h{}.txt'.format(dt_string), 'w')
# f.close()
# with open('hyperparameters/h{}.txt'.format(dt_string), 'w') as convert_file:
#     convert_file.write(json.dumps(hyperparameters))

# # Sauvegarde des erreurs
# errors = {"errorMin": rhoMin, "errorMax": rhoMax}
# f = open('errors/er{}.txt'.format(dt_string), 'w')
# f.close()
# with open('errors/er{}.txt'.format(dt_string), 'w') as convert_file:
#     convert_file.write(json.dumps(errors))
